In [0]:
%%sh
aws s3 ls s3://aardvark-prod-dca-data/oss/GAME_CROSS_GENRE_KPI/version=1.0.0/range_type=MONTH/date=2019-07-31/
 



 

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.genre-cross.v1/fact/granularity=monthly/date=2019-07-31/part-00000-37b7762c-4c94-4290-a335-d1250c347d15.c000.gz.parquet").show()

spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/genre_report/version=1.0.0/range_type=MONTH/date=2019-08-31/").show()

In [0]:

from applications.db_check_v1.common.constants import query, citus_settings

sql = "select date, count(1) from aso.aso_sov_search_ads_keyword_fact_v1 where date in (select distinct date FROM aso.aso_sov_search_ads_keyword_fact_v1  where granularity='daily' order by date desc ) group by date order by date;"

print query(citus_settings("aa"), sql)


In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

source_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/'
dest_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_month.v3/date={}'

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review.v1/fact/process_granularity=monthly/", depth_is_1=True)
print path_list[155]
for path in path_list[155:]:
    print path.split("/")[4].split("=")[1]

    count_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).where(
        "market_code in ('apple-store','google-play')").select("review_id").distinct().count()
    new_path = path.split("/")[4].split("=")[1]
    df = spark.read.json(dest_path.format(new_path)).cache()
    count_2 = df.filter("index is not null").count()
    count_3 = df.filter("_identifier is not null").count()

    # print count_1
    if count_1 == count_2 == count_3:
        print 'pass', path, count_1
        pass
        # print 'pass', path
    else:
        print "not equal!!!!!!", count_1, count_2, count_3, path


In [0]:
%%sh
PGPASSWORD='90H1O7a9c2PQj7SC' psql -h b2b-prod-datapipeline-meta-new.crlexxwtzodp.us-east-1.rds.amazonaws.com  -U aa_datapipeline -d pipeline_meta -p 5432 << EOF
select count(1), status from review_load_queue group by status;
select (max(update_time) - min(update_time)) / count(update_time) as avg_time_per_file from review_load_queue where create_time > '2020-02-11 03:00:00' and status = 'finished';
EOF

In [0]:

from pyspark.sql.functions import desc

dup_list = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2017-06/').select('review_id','process_date','process_hour').distinct().groupBy("review_id").agg({"*":"count"}).where("count(1)>1").orderBy("count(1)",ascending = False).take(10)

for x in dup_list:
    result1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2017-06/').filter("review_id='{}'".format(x["review_id"])).groupBy("process_date","process_hour","time").agg({"*":"count"}).orderBy(desc("time"),desc("process_date"),desc("process_hour")).take(1)
    result2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month=2017-06/").filter("review_id='{}'".format(x["review_id"])).take(1)
    print result1,result2
    if result1[0]["process_date"]==result2[0]["process_date"] and result1[0]["process_hour"]==result2[0]["process_hour"] and result1[0]["time"]==result2[0]["time"]  :
        print 'pass' , x 
    else:
        print 'failed!!!',result1, result2,x


In [0]:

print sc.parallelize([1,2,3,4,5], 4).cache().map(lambda x:x+1).collect()

In [0]:


test_list = [['Hello', 'aworld'], ['I', 'am', 'fine']]
from pyspark.sql import Row

df =spark.createDataFrame(list(map(lambda x : Row(words=x, anther='aha'), test_list)))
df.show()

print dir(df)
print vars(df)
print hasattr(df,'name')


In [0]:

review_row=set()
review_row.add(1)
review_row.add(2)

df=spark.createDataFrame([('2020-05-01', 'daily', len(review_row))],schema=["process_date","granularity","agg_count"] )
df.coalesce(1).write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.review.count/test.parquet",mode="append",  partitionBy=["process_date"])

# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.review.count/process_date=2020-05-01/part-00001-7c8e471b-0072-4031-9595-e27070423ad5.c000.snappy.parquet").show()


print dir(df.write)


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.review.count/process_date=2020-03-13/part-00000-c640f0dc-7a8c-4f2b-93b3-808a86c4be5d.c000.snappy.parquet | sort -n


In [0]:

print spark.read.parquet("s3://aardvark-prod-dca-data/oss/GAME_CROSS_GENRE_KPI/version=1.0.0/range_type=MONTH/date=2019-07-31/").show() #groupBy("metric_name").count().show(20,False)


# +------------------------------+-----+
# |metric_name                   |count|
# +------------------------------+-----+
# |shared_users                  |95247|
# |related_category_UP           |95247|
# |XGA                           |95247|
# |selected_category_active_users|95247|
# |selected_threshold            |95247|
# |XUP                           |95247|
# +------------------------------+-----+

# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.review.count/process_date=2020-03-13/part-00000-1f4aa426-7ecd-492d-a818-90c710966871.c000.snappy.parquet").show()